In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pyplot as pylab
import seaborn as sns

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import re
import keras

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU
from keras.layers import Bidirectional, GlobalMaxPool1D, Conv1D, GlobalAveragePooling1D, concatenate
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import CSVLogger, ReduceLROnPlateau, ModelCheckpoint 
from keras.models import Sequential

In [0]:
train=pd.read_csv('train.csv')
pd.isnull(train).sum()
# Read train data
#train = pd.read_csv('train.csv')
train.dropna(inplace=True)

list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

y_train = train[list_classes]

FileNotFoundError: ignored

In [0]:
test=pd.read_csv('test.csv')
pd.isnull(test).sum()

In [0]:
classes=list(train.columns.values)[2:]

In [0]:
# the maximum number of words considered is 100000
MAX_NB_WORDS = 100000
# the size of the sentences will be 250
max_seq_len = 250


In [0]:
def func(change):
    old=change[0]
    new=change[1]
    train['comment_text']=train['comment_text'].replace(old, new)
    test['comment_text']=test['comment_text'].replace(old, new)

In [0]:
changes=[('\'ll', ' will'), ('don\'t', ' do not'), ('\'ve', ' have'), ('don\'t', ' do not'), ('aren\'t', ' are not'), ('won\'t', ' will not'), ('wont', ' will not'), ('can\'t', ' cannot'), ('cant', ' cannot'), ('shan\'t', ' shall not'), ('shant', ' shall not'), ('\'m', ' am'), ("doesn't", "does not"), ("doesnt", "does not"), ( "didn't", "did not"), ( "didnt", "did not"), ("hasn't", "has not"), ("hasnt", "has not"), ("haven't", "have not"), ("havent", "have not"), ("wouldn't", "would not"), ( "didn't", "did not"), ( "didnt", "did not"), ("it's" , "it is"), ( "that's" , "that is"), ("weren't" , "were not"), ("werent" , "were not"), (' u ', ' you '), (' U ', ' you ')]

In [0]:
for x in changes:
    func(x)

In [0]:
for data in [train, test]:
    
    data['comment_text'] = data['comment_text'].apply(lambda x: re.sub('[\(\)\"\t_\n.,:=!@#$%^&*-/[\]?|1234567890—]', ' ', x).strip())

In [0]:
train_word_tokens = train['comment_text'].apply(lambda x: word_tokenize(x))
test_word_tokens = test['comment_text'].apply(lambda x: word_tokenize(x))

In [0]:
processed_train=list()
for x in train_word_tokens:
    new=[l for l in x if l not in stopwords.words('english')]
    processed_train.append(' '.join(new))
    
processed_test=list()
for x in test_word_tokens:
    new=[l for l in x if l not in stopwords.words('english')]
    processed_test.append(' '.join(new))

In [0]:
tokenizer = Tokenizer(num_words=100000, lower=True)
temp=processed_train + processed_test
tokenizer.fit_on_texts(temp)  #leaky
ws_train = tokenizer.texts_to_sequences(processed_train)
ws_test = tokenizer.texts_to_sequences(processed_test)
word_i = tokenizer.word_index



ws_train = sequence.pad_sequences(ws_train, maxlen=250)
ws_test = sequence.pad_sequences(ws_test, maxlen=250)



In [0]:
embedd = 300

#load embeddings
print('loading first word embeddings')
ei = {}
p = codecs.open('glove.840B.300d.txt', encoding='utf-8')

for line in tqdm(p):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    ei[word] = coefs
p.close()
print('found %s word vectors' % len(ei))




#embedding matrix
print('preparing for embedded matrix')
word_nthere = []
nb_words = min(MAX_NB_WORDS, len(word_i))
emg = np.zeros((nb_words, embedd))
for word, i in word_i.items():
    if i >= nb_words:
        continue
    ev = ei.get(word)
    if (ev is not None) and len(ev) > 0:
        # words not found in embedding index will be all-zeros.
        emg[i] = ev
    else:
        word_nthere.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(emg, axis=1) == 0))

loading first word embeddings


NameError: ignored

In [0]:
print('loading second word embedding')
EMBEDDING_FILE = 'crawl-300d-2M.vec'

def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
ei = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE, encoding="utf8"))

word_i = tokenizer.word_i
emc = np.zeros((nb_words, embedd))
for word, i in word_i.items():
    if i >= MAX_NB_WORDS: continue
    ev = ei.get(word)
    if ev is not None: emc[i] = ev


del ei







loading second word embeddings...


FileNotFoundError: ignored

In [0]:
# Creating the model using lstm

inp = Input(shape=(max_seq_len, ))
emb_glove = Embedding(nb_words, 300, 
                weights=[emg], input_length = max_seq_len, trainable=False)(inp)

emb_crawl = Embedding(nb_words, 300,
                weights=[emc], input_length = max_seq_len, trainable=False)(inp)

conc1 = concatenate([emb_glove, emb_crawl])
x = Bidirectional(LSTM(400, return_sequences=True))(conc1)
x = Bidirectional(GRU (400, return_sequences=True))(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
out = concatenate([avg_pool, max_pool])

out = Dense(200, activation="relu")(out)
out = Dense(y_train.shape[1], activation="sigmoid")(out)

rdel = Model(inputs=inp, outputs=out)




In [0]:
rdel.compile(loss='binary_crossentropy', optimizer = keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])


In [0]:
rdel.summary()

In [0]:
rdel.fit(ws_train, y_train, epochs=6, batch_size=80, shuffle=True, validation_split=0.1,verbose = 2)

In [0]:
y_test = rdel.predict(ws_test)

NameError: ignored

In [0]:
ss = pd.read_csv("sample_submission.csv")

ss[list_classes] = y_test
ss.to_csv("results.csv", index=False)